In [1]:

# 2. （可选）挂载 Google Drive，如果你的数据集存放在 Drive 上
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [16]:
!rm -rf /content/augmented_images

In [18]:

# 2. 安装并导入必要的库
!pip install torchvision --quiet

import os
import random
from PIL import Image
import torch
from torchvision import transforms
from google.colab import files

# 3. 定义源数据路径和保存路径
src_dir  = '/content/drive/MyDrive/DiT_Project/dataset/Bee_1000Data'
save_dir = '/content/augmented_norm_then_denorm'
os.makedirs(save_dir, exist_ok=True)

# 4. Define normalization and denormalization transforms
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std =[0.229, 0.224, 0.225]
)

# Denormalization: x_denorm = x_norm * std + mean
denormalize = transforms.Normalize(
    mean=[-m/s for m, s in zip([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])],
    std =[1.0/s for s in [0.229, 0.224, 0.225]]
)

# 5. Define augmentation pipeline including normalization
transform_norm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    normalize,
])

# 6. 列出所有源图像文件
files_list = [f for f in os.listdir(src_dir)
              if f.lower().endswith(('.png','.jpg','.jpeg'))]
if not files_list:
    raise RuntimeError(f"No images found in {src_dir}")

# 7. 循环生成 1000 张：先 normalize 再 denormalize，再保存
to_pil = transforms.ToPILImage()
for i in range(1000):
    fn  = random.choice(files_list)
    img = Image.open(os.path.join(src_dir, fn)).convert('RGB')
    t   = transform_norm(img)              # 归一化后的 Tensor
    t   = denormalize(t).clamp(0, 1)       # 反归一化并截断到 [0,1]
    out = to_pil(t)                        # 转回 PIL.Image
    out.save(os.path.join(save_dir, f'aug_{i:04d}.jpg'))

print(f"Done! Generated {len(os.listdir(save_dir))} images → {save_dir}")

# 8. 压缩并下载到本地
zip_path = '/content/augmented_norm_then_denorm.zip'
!zip -r "{zip_path}" "{save_dir}"
files.download(zip_path)

Done! Generated 1000 images → /content/augmented_norm_then_denorm
  adding: content/augmented_norm_then_denorm/ (stored 0%)
  adding: content/augmented_norm_then_denorm/aug_0243.png (deflated 0%)
  adding: content/augmented_norm_then_denorm/aug_0483.png (deflated 0%)
  adding: content/augmented_norm_then_denorm/aug_0217.png (deflated 0%)
  adding: content/augmented_norm_then_denorm/aug_0944.png (deflated 0%)
  adding: content/augmented_norm_then_denorm/aug_0693.png (deflated 0%)
  adding: content/augmented_norm_then_denorm/aug_0689.png (deflated 0%)
  adding: content/augmented_norm_then_denorm/aug_0000.png (deflated 0%)
  adding: content/augmented_norm_then_denorm/aug_0124.png (deflated 0%)
  adding: content/augmented_norm_then_denorm/aug_0008.png (deflated 0%)
  adding: content/augmented_norm_then_denorm/aug_0489.png (deflated 0%)
  adding: content/augmented_norm_then_denorm/aug_0436.png (deflated 0%)
  adding: content/augmented_norm_then_denorm/aug_0948.png (deflated 0%)
  adding: co

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>